<h1>Sentiment Classification Using LogisticRegression - IMDB Reviews</h1>

In [52]:
import torch 
from torch import nn
import numpy as np
import pandas as pd
import spacy
import re
from sklearn.model_selection import train_test_split

In [53]:
#Loading the dataset
dataSet=pd.read_csv(r"D:\Datasets\IMDB Dataset.csv")
dataSet.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


<h2> Preprocessing

In [54]:
#Check for any null  values
print(pd.unique(dataSet['sentiment']))
print(dataSet.isna().sum())

['positive' 'negative']
review       0
sentiment    0
dtype: int64


In [55]:
#Turn The sentiment into numbers

label_mapping={'positive':1,'negative':0}

dataSet['sentiment'] = dataSet['sentiment'].map(label_mapping)
dataSet.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [78]:
#Create Positive and negative dicts
global nlp
nlp=spacy.load("en_core_web_sm")
global pos_dict,neg_dict
pos_dict,neg_dict={},{}
#Pre train the pos and neg dicts

with open("Positive.txt" ,'r') as pos:
    pos_list=pos.read().split()
with open("Negative.txt",'r') as neg:
    neg_list=neg.read().split()

for i in pos_list:
    pos_dict[i]=0
for i in neg_list:
    neg_dict[i]=0

    

0


In [143]:
#Define Methods for cleaning , processing and extracting features from the data


def remove_stops(arr):
    
    stop_words = nlp.Defaults.stop_words
    doc=nlp(arr)
    cleaned_tokens = [token.text for token in doc if token.text not in stop_words and not token.is_punct]
    doc=nlp(" ".join(cleaned_tokens))
    cleaned_tokens=[token.lemma_ for  token in doc]
    return cleaned_tokens


def apply_regex(arr):

    pattern=r"@\S+|https?:\/\/\S+|www\.\S+|<br\s*/?>"
    cleaned_text=re.sub(pattern,'',arr)
    return cleaned_text

def extract_feature_vector(arr):
    words=arr.split()
    for word in words:
        if word in pos_dict:
            pos_dict[word] += 1
        elif word in neg_dict:
            neg_dict[word] += 1
    
    pos_sum = sum(pos_dict.values())
    neg_sum = sum(neg_dict.values())
    for i in pos_list:
        pos_dict[i]=0
    for i in neg_list:
        neg_dict[i]=0
    
    return np.array([1, pos_sum, neg_sum])
print(dataSet['review'][9])
text=remove_stops(dataSet['review'][9])
text=apply_regex(" ".join(text))
extract_feature_vector(text)



    




If you like original gut wrenching laughter you will like this movie. If you are young or old then you will love this movie, hell even my mom liked it.<br /><br />Great Camp!!!


array([1, 4, 1])

In [155]:
#Split the data into training and testing sets
train_x,test_x,train_y,test_y=train_test_split(list(dataSet['review']),list(dataSet['sentiment']),random_state=3,train_size=5000,test_size=0.33)
for i in range(len(train_x)):
    text=remove_stops(train_x[i])
    text=apply_regex(" ".join(text))
    train_x[i]=extract_feature_vector(text)
        





In [156]:
train_y_1=torch.tensor(train_y).reshape(5000,1)
test_y=torch.tensor(test_y)
train_x=torch.tensor(train_x).squeeze().float()
train_y_1

tensor([[1],
        [1],
        [1],
        ...,
        [1],
        [0],
        [0]])

In [157]:
#Build the model
class LogisticRegression(nn.Module):
    def __init__(self,lr,epochs,feature_no):
        super().__init__()
        self.epoch=epochs
        self.l1=nn.Linear(feature_no,1)
        self.optimizer=torch.optim.SGD(self.parameters(),momentum=0.5,lr=lr)
        self.loss=nn.BCELoss()
        self.activation=nn.Sigmoid()

    def forward(self,x):
        return self.activation(self.l1(x))

    def fit(self,x,y):
        for i in range(self.epoch):
            y_pred=self.forward(x)
            loss=self.loss(y_pred,y)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
    

In [158]:
#Train the model
model=LogisticRegression(0.01,1000,3)
model.fit(train_x,train_y_1.float())





In [160]:
#Testing


        

16500


ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'numpy.ndarray'>